Importing all needed libraries

In [ ]:
import pickle as pk
import xgboost as xgb
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_predict, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error


Reading the data from CSV files

In [ ]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Intro to AI/players_21.csv')

In [ ]:
dataset = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Intro to AI/players_22.csv')

<ipython-input-6-822e9eed5024>:1: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Intro to AI/players_22.csv')


Displaying the first 5 rows of the data to have an idea of what the data looks like

---



In [ ]:
data.head()

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,103500000.0,560000.0,33,...,52+3,52+3,52+3,62+3,19+3,https://cdn.sofifa.net/players/158/023/21_120.png,https://cdn.sofifa.net/teams/241/60.png,https://cdn.sofifa.net/flags/es.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",92,92,63000000.0,220000.0,35,...,54+3,54+3,54+3,61+3,20+3,https://cdn.sofifa.net/players/020/801/21_120.png,https://cdn.sofifa.net/teams/45/60.png,https://cdn.sofifa.net/flags/it.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
2,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,91,91,111000000.0,240000.0,31,...,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/21_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,NaN,https://cdn.sofifa.net/flags/pl.png
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,132000000.0,270000.0,28,...,49+3,49+3,49+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/21_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CAM, CM",91,91,129000000.0,370000.0,29,...,69+3,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/21_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png


Checking the type of values for the 'overall' column

----


In [ ]:
data['overall']

0        93
1        92
2        91
3        91
4        91
         ..
18939    47
18940    47
18941    47
18942    47
18943    47
Name: overall, Length: 18944, dtype: int64

Creating a list of columns to encode the categorical features using one-hot encoding in the dataset and concatenating with to the numerical data.

In [ ]:
columns_to_encode = []
for column in data.columns:
  if data[column].dtype == object:
    columns_to_encode.append(column)
# Use get_dummies with dummy_na=True
df_encoded= pd.get_dummies(columns_to_encode, prefix = "encoded_", dummy_na=True)
data.drop(columns_to_encode, axis = 1, inplace = True)
data = pd.concat([data, df_encoded], axis=1)


Iterating through each column in the DataFrame 'data' and determining whether the percentage of missing values in the column exceeds 30% of the total number of rows. If it exceeds the threshold, remove the column from the 'data' DataFrame.This technique is used to delete columns that have a significant percentage of missing data and may be useless for analysis.


In [ ]:
for column in data.columns:
    if data[column].isna().sum() > 0.3 * len(data):
        data.drop(column, axis=1, inplace=True)

Storing the list of column names

In [ ]:
columns = data.columns

Create a SimpleImputer object called 'imputer


In [ ]:
imputer = SimpleImputer()

Converted column names to strings for consistency

In [ ]:
data.columns = data.columns.astype(str)

Imputing the data to handle missing values


In [ ]:
imputer.fit(data)
data =imputer.fit_transform(data)
data = pd.DataFrame(data, columns = columns)

Creating new subset columns in data by calculating the mean of specific groups of columns by aggregating related features into average values for further analysis or modeling.

In [ ]:
data['goalkeeping_average'] = data[['goalkeeping_diving', 'goalkeeping_handling', 'goalkeeping_kicking', 'goalkeeping_positioning', 'goalkeeping_reflexes']].mean(axis =1)
data['attacking'] = data[['attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy',
                                'attacking_short_passing', 'attacking_volleys']].mean(axis =1)
data['defending_average'] = data[['defending_marking_awareness', 'defending_standing_tackle', 'defending_sliding_tackle' ]].mean(axis =1)
data['skill'] = data[['skill_dribbling','skill_curve','skill_fk_accuracy','skill_long_passing','skill_ball_control']].mean(axis =1)
data['power_average'] = data[['power_shot_power', 'power_jumping', 'power_stamina',
                                'power_strength', 'power_long_shots' ]].mean(axis =1)
data['movement_average'] = data[['movement_acceleration', 'movement_sprint_speed', 'movement_agility',
                                'movement_reactions', 'movement_balance']].mean(axis =1)
data['goalkeeping_average'] = data[['goalkeeping_diving', 'goalkeeping_diving', 'goalkeeping_kicking',
                                'goalkeeping_positioning', 'goalkeeping_reflexes','goalkeeping_handling']].mean(axis =1)
data['wage_to_value_ratio'] = data['value_eur'] / data['wage_eur']


Using correlation to determine the features that have a strong relationship
with the target("overall")

In [ ]:
correlations = data.corrwith(data['overall'])
correlation_pairs = list(zip(data.columns, correlations)) # A list of tuples containing column names and their correlation coefficients with the target variable.


This code filters columns from the DataFrame 'data' based on their correlation with the target variable.
It removes columns where the correlation coefficient with the target variable is less than 0.4.

In [ ]:
for item in correlation_pairs:
  column_name, value = item
  if value <0.4:
    data.drop([column_name],axis =1, inplace = True)

Dropping the features that are repeated. These features individual have a correlation above 0.4 with the target variable. However, the subset feature has a better relation with the target.

In [ ]:
data.drop(['skill_curve', 'skill_long_passing', 'skill_ball_control','attacking_crossing', 'attacking_short_passing','power_shot_power',
           'power_long_shots'], axis = 1, inplace = True)

In [ ]:
data.columns

Index(['overall', 'potential', 'value_eur', 'wage_eur', 'age',
       'international_reputation', 'release_clause_eur', 'shooting', 'passing',
       'dribbling', 'physic', 'movement_reactions', 'mentality_aggression',
       'mentality_vision', 'mentality_composure', 'attacking', 'skill',
       'power_average'],
      dtype='object')

Scaling the data using MinMaxScaler

In [ ]:
y = data['overall']
x = data.drop('overall', axis = 1)

In [ ]:
scaler = MinMaxScaler()
columns_x = x.columns
x = scaler.fit_transform(x)
x =pd.DataFrame(x, columns = columns_x)

Saving the scaler model for future use


In [ ]:
with open('/content/drive/MyDrive/scaler_model2.pkl', 'wb') as scaler_file:
    pk.dump(scaler, scaler_file)

Defining the learning models that will be trained

In [ ]:
rfr = RandomForestRegressor()
dtr = DecisionTreeRegressor()
gbr = GradientBoostingRegressor()
xgb= XGBRegressor()

In [ ]:
models = {
    "Random Forest": rfr,
    "Decision Tree": dtr,
    "Gradient Boost": gbr,
    "XGBoost": xgb
}
mae_score= {} # a dictionary to store the models and a list of their mean absolute error

Set up k-fold cross-validation with 5 folds, shuffling the data, and using a random state of 42.Storing mean absolute error values in a list to keep results from cross validation.

In [ ]:
# Set up k-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

Loop through different models and perform k-fold cross-validation for each model, then split the data into training and testing sets for each fold using 'kfold'
Fit the current model on the training data
Make predictions on the test data
Calculate Mean Absolute Error (MAE) between true and predicted values
Store the MAE scores in the'mae_scores', for each model
Store the final MAE scores for each model in the 'mae_score' dictionary


In [ ]:
for model_name, model in models.items():
  # Lists to store the cross-validation results
    mae_scores = []
    for train_index, test_index in kfold.split(x):
        X_train, X_test = x.iloc[train_index], x.iloc[test_index]
        Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]

        # Fit the model on the training data
        model.fit(X_train, Y_train)

        # Make predictions on the test data
        y_pred = model.predict(X_test)

        # Calculate mean squared error and store it
        mae = mean_absolute_error(Y_test, y_pred)
        mae_scores.append(mae)
    mae_score[model_name] = mae_scores


Display the mean absolute error for each model

In [ ]:
print(mae_score)

{'Random Forest': [0.2603879651623121, 0.2618527315914492, 0.2514198997096861, 0.26256268144629213, 0.27096092925026405], 'Decision Tree': [0.28186856690419637, 0.2755344418052256, 0.25864344154130375, 0.27606228556347323, 0.2866948257655755], 'Gradient Boost': [0.5841377183803714, 0.5828572882674236, 0.5976370062963277, 0.5925029175697943, 0.5991353232803779], 'XGBoost': [0.34894021427640814, 0.3414505641169081, 0.35789291812738666, 0.35043861306604024, 0.36425927473348196]}


Finding the best model with the lowest mean absolute error 'min_error' is initialized to positive infinity and 'best_model' to None.
Loop through the models and calculate the mean error for each using k-fold cross-validation. If the mean error for this model is smaller than the current minimum error, update the best model and min_error.

In [ ]:
min_error = float('inf')
best_model = None

for model_name, error_list in mae_score.items():
    mean_error = sum(error_list) / len(error_list)

    if mean_error < min_error:
        min_error = mean_error
        best_model = model_name

Store the name of the best model.

In [ ]:
model = models[best_model]

Display the columns of the 'data' dataframe.

In [ ]:
data.columns

Index(['overall', 'potential', 'value_eur', 'wage_eur', 'age',
       'international_reputation', 'release_clause_eur', 'shooting', 'passing',
       'dribbling', 'physic', 'movement_reactions', 'mentality_aggression',
       'mentality_vision', 'mentality_composure', 'attacking', 'skill',
       'power_average'],
      dtype='object')

Display summary statistics for the 'data' dataframe.

In [ ]:
data.describe() # description of features

,overall,potential,value_eur,wage_eur,age,international_reputation,release_clause_eur,shooting,passing,dribbling,physic,movement_reactions,mentality_aggression,mentality_vision,mentality_composure,attacking,skill,power_average
count,18944.000000,18944.000000,1.894400e+04,18944.000000,18944.000000,18944.000000,1.894400e+04,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000
mean,65.677787,71.086729,2.902288e+06,9148.482825,25.225823,1.091850,4.296353e+06,52.274954,57.139434,62.455430,64.458988,61.609428,55.488862,53.828231,57.978674,49.731704,51.245133,59.288207
std,7.002278,6.109985,7.695181e+06,19774.654223,4.697354,0.361841,9.791922e+06,13.199766,9.692357,9.480136,9.195223,9.112954,17.203045,13.724072,12.118390,14.893365,15.753738,10.174109
min,47.000000,47.000000,9.000000e+03,500.000000,16.000000,1.000000,9.000000e+03,16.000000,25.000000,25.000000,28.000000,24.000000,9.000000,9.000000,12.000000,8.400000,8.000000,24.400000
25%,61.000000,67.000000,4.750000e+05,1000.000000,21.000000,1.000000,5.440000e+05,44.000000,51.000000,58.000000,59.000000,56.000000,44.000000,45.000000,50.000000,44.400000,44.200000,52.800000
50%,66.000000,71.000000,9.750000e+05,3000.000000,25.000000,1.000000,1.200000e+06,52.274954,57.139434,62.455430,64.458988,62.000000,58.000000,55.000000,59.000000,52.400000,53.800000,60.400000
75%,70.000000,75.000000,2.100000e+06,9000.000000,29.000000,1.000000,4.296353e+06,62.000000,63.000000,68.000000,71.000000,68.000000,68.000000,64.000000,66.000000,59.400000,62.000000,66.800000
max,93.000000,95.000000,1.855000e+08,560000.000000,53.000000,5.000000,2.031000e+08,93.000000,93.000000,95.000000,91.000000,95.000000,96.000000,95.000000,96.000000,87.400000,94.000000,88.800000


Splitting the data into training and testing sets.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

Fine tuning the gradient boosting regressor, performed grid search with cross-validation to find the best hyperparameters for the model.Retrieved the best model with the optimal hyperparameters then evaluated tofine tuned model on the test set

In [ ]:
param_grid = {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20]}

# Assuming 'model' and 'param_grid' are defined earlier
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_absolute_error')
grid_search.fit(x_train,y_train)

# Now, you can use these best parameters to train your model on the entire training set or do further analysis
best_model = grid_search.best_estimator_
best_model.fit(x_train, y_train)

# Evaluate the model on the test set
y_pred = best_model.predict(x_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error on Test Set: {mae}')



Mean Absolute Error on Test Set: 0.2568670432534243


In [ ]:
grid_search.best_params_

{'max_depth': 20, 'n_estimators': 200}

In [ ]:
print(best_model)

RandomForestRegressor(max_depth=20, n_estimators=200)


Created an empty dictionary to store the best hyperparameters.

In [ ]:
best_params_grid = {}

Display the minimum error value calculated earlier.


In [ ]:
min_error

0.26143684143200063

Checking if the tuned model is better than the actual mode:We check whether the Mean absolute error of the tuned model is smaller than that of the original

In [ ]:
mae < min_error

True

Display the columns of the 'data' dataframe.

In [ ]:
data.columns

Index(['overall', 'potential', 'value_eur', 'wage_eur', 'age',
       'international_reputation', 'release_clause_eur', 'shooting', 'passing',
       'dribbling', 'physic', 'movement_reactions', 'mentality_aggression',
       'mentality_vision', 'mentality_composure', 'attacking', 'skill',
       'power_average'],
      dtype='object')

Saving the best model in a file

In [ ]:
file_name = "/content/drive/My Drive/Colab Notebooks/best_model2.pkl"
with open(file_name, "wb") as file:
  pk.dump(best_model,file)

Testing my model with Player22.
Loop through the dataset and remove columns from the 'dataset' dataframe with too many missing values.

In [ ]:
for column in dataset.columns:
    if dataset[column].isna().sum() > 0.3 * len(data):
        dataset.drop(column, axis=1, inplace=True)

To determine which columns in the 'dataset' DataFrame have a data type of 'object' (which usually indicates categorical variables).
The names of these categorical columns should be saved in the 'columns_to_encode' list.
To handle missing data, use the 'pd.get_dummies' function to one-hot encode the category columns with 'dummy_na=True'.
Along the columns (axis=1), concatenate the one-hot encoded columns with the original 'dataset'.
Using 'drop', implace, and axis=1, remove the original categorical columns from 'dataset'.

In [ ]:
columns_to_encode = []
for column in dataset.columns:
  if dataset[column].dtype == object:
    columns_to_encode.append(column)
# Use get_dummies with dummy_na=True
df_encoded= pd.get_dummies(columns_to_encode, columns=columns_to_encode, dummy_na=True)
dataset = pd.concat([dataset, df_encoded], axis=1)
dataset.drop(columns_to_encode, axis = 1, inplace = True)

Ensure all columns are of string data type

In [ ]:
dataset.columns = dataset.columns.astype(str)
columns = dataset.columns

Fitting the imputer to the 'dataset' and imputing missing values.

In [ ]:
imputer.fit(dataset)
dataset =imputer.fit_transform(dataset)
dataset= pd.DataFrame(dataset, columns = columns)

Create new columns in 'dataset' by calculating the mean of specific groups of columns, similar to what was done for the 'data' dataset.

In [ ]:
dataset['goalkeeping_average'] = dataset[['goalkeeping_diving', 'goalkeeping_handling', 'goalkeeping_kicking', 'goalkeeping_positioning', 'goalkeeping_reflexes']].mean(axis =1)
dataset['attacking'] = dataset[['attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy',
                                'attacking_short_passing', 'attacking_volleys']].mean(axis =1)
dataset['defending_average'] = dataset[['defending_marking_awareness', 'defending_standing_tackle', 'defending_sliding_tackle' ]].mean(axis =1)
dataset['skill'] = dataset[['skill_dribbling','skill_curve','skill_fk_accuracy','skill_long_passing','skill_ball_control']].mean(axis =1)
dataset['power_average'] = dataset[['power_shot_power', 'power_jumping', 'power_stamina',
                                'power_strength', 'power_long_shots' ]].mean(axis =1)
dataset['movement_average'] = dataset[['movement_acceleration', 'movement_sprint_speed', 'movement_agility',
                                'movement_reactions', 'movement_balance']].mean(axis =1)
dataset['goalkeeping_average'] = dataset[['goalkeeping_diving', 'goalkeeping_diving', 'goalkeeping_kicking',
                                'goalkeeping_positioning', 'goalkeeping_reflexes','goalkeeping_handling']].mean(axis =1)

Filtering the 'dataset' to keep only the features used to train the model.

In [ ]:
columns_to_keep = data.columns
dataset = dataset[columns_to_keep]

Scaling the test data using the previous MinScaler

In [ ]:
y= dataset['overall']
x = dataset.drop('overall', axis =1)
columns_x = x.columns
x = scaler.fit_transform(x)
x =pd.DataFrame(x, columns = columns_x)

load the model from the file.

In [ ]:
loaded_model = pk.load(open(file_name, 'rb'))

Use the loaded model to make predictions on the test data.

In [ ]:
prediction = loaded_model.predict(x)

Calculate the mean absolute error of the predictions.

In [ ]:
mae = mean_absolute_error(y, prediction)
print(f"Mean Absolute Error: {mae}")

Mean Absolute Error: 0.819514639586034
